# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
# импорт библиотеки pandas, чтение файла с данными, сохранение в credits, 
# вывод на экран первых 10 строк таблицы credits и получение общей информации о данных в таблице credits
import pandas as pd
credits = pd.read_csv('/datasets/data.csv')
display(credits.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
# получение общей информации о данных в таблице credits
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# замена отрицательных значений в колонке "Общий трудовой стаж в днях" с помощью метода .abs(), 
# вывод первых 10 строк на экран
credits['days_employed'] = credits['days_employed'].abs() 
display(credits.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В таблице "Статистика о платежеспособности клиентов" 21525 строк и 12 столбцов. Тип данных в двух столбцах - float64, в 5 столбцах - int64 и ещё в пяти - object.  
В столбце "Общий трудовой стаж" есть отрицательные значения, что противоречит логике, т.к. данные в этом столбце измеряются в днях и могут быть только положительными. Возможно такая ошибка возникла при копировании, считывании или смене формата данных, т.е. носит технологический характер. Мы привели данные в порядок - взяли значения в столбце "Общий трудовой стаж" по модулю, чтобы можно было с ними работать.  
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Предобработка данных

### Пропуски значений

In [4]:
# подсчёт пропусков
credits.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения в столбцах "Общий трудовой стаж" и "Ежемесячный доход" составляют около 10%, это значительно может повлиять на результаты исследований. Данные из колонки "Ежемесячный доход" напрямую влияют на способность погашать кредит, поэтому проанализируем тех клиентов, которые не указали данные о своих доходах.

In [5]:
# поиск строк с пропусками в столбце total_income и вывод на экран первых десяти
credits_filtered = credits[credits['total_income'].isna()]
credits_filtered.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


В столбцах "Общий трудовой стаж" и "Ежемесячный доход" обнаружены пропущенные значения Nan. Предположим, что пропущенные значения вызваны человеческим фактором - возможно некоторые клиенты имеют неподтвержденные доходы или безработные. Посмотрим, клиентов с каким типом занятости больше всего среди тех, кто не указал доход.

In [6]:
# группировка строк с пропущенными значениями по типу занятости и подсчет количества клиентов для каждого типа 
credits_filtered.groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Большинство клиентов без дохода и стажа оказались сотрудниками банка. Безработными их назвать сложно, поэтому возьмем медианную зарплату сотрудников и заменим пропущенные значения в столбце "Ежемесячный доход" для тех, кто работает в банке и точно делает это не бесплатно

In [7]:
# подсчёт медианной зарплаты для сотрудников
employee_income_avg = credits[credits['income_type'] == 'сотрудник']['total_income'].mean()
print(employee_income_avg)

161380.26048788553


In [8]:
# замена пропущенных значений для сотрудников в столбце "Ежемесячный доход" на медианную зарплату сотрудника
credits.loc[(credits['income_type'] == 'сотрудник')&(credits['total_income'].isna()), 'total_income'] = employee_income_avg

# проверка корректности замены пропущенных значений
credits[credits['total_income'].isna()].groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
компаньон          508
пенсионер          413
госслужащий        147
предприниматель      1
Name: income_type, dtype: int64

In [9]:
# подсчёт медианной пенсии
pension_avg = credits[credits['income_type'] == 'пенсионер']['total_income'].mean()
print(pension_avg)

137127.4656901654


In [10]:
# замена пропущенных значений для пенсионеров в колонке "Ежемесячный доход" на медианный размер пенсии
credits.loc[(credits['income_type'] == 'пенсионер')&(credits['total_income'].isna()), 'total_income'] = pension_avg

# проверка корректности замены пропущенных значений
credits[credits['total_income'].isna()].groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
компаньон          508
госслужащий        147
предприниматель      1
Name: income_type, dtype: int64

In [11]:
# подсчёт медианного дохода компаньона
companion_income_avg = credits[credits['income_type'] == 'компаньон']['total_income'].mean()
print(companion_income_avg)

202417.4614617771


In [12]:
# замена пропущенных значений для компаньонов в колонке "Ежемесячный доход" на медианный доход компаньона
credits.loc[(credits['income_type'] == 'компаньон')&(credits['total_income'].isna()), 'total_income'] = companion_income_avg

# проверка корректности замены пропущенных значений
credits[credits['total_income'].isna()].groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
госслужащий        147
предприниматель      1
Name: income_type, dtype: int64

In [13]:
# подсчёт медианного дохода госслужащего
state_employee_income_avg = credits[credits['income_type'] == 'госслужащий']['total_income'].mean()
print(state_employee_income_avg)

170898.30992266268


In [14]:
# замена пропущенных значений для компаньонов в колонке "Ежемесячный доход" на медианный доход компаньона
credits.loc[(credits['income_type'] == 'госслужащий')&(credits['total_income'].isna()), 'total_income'] = state_employee_income_avg

# проверка корректности замены пропущенных значений
credits[credits['total_income'].isna()].groupby('income_type')['income_type'].count().sort_values(ascending=False)

income_type
предприниматель    1
Name: income_type, dtype: int64

In [15]:
# подсчёт медианного дохода предпринимателя
businessman_employee_income_avg = credits[credits['income_type'] == 'предприниматель']['total_income'].mean()
print(businessman_employee_income_avg)

499163.1449470857


In [16]:
# замена пропущенного значения для предпринимателя в колонке "Ежемесячный доход" на медианный доход предпринимателя
credits.loc[(credits['income_type'] == 'предприниматель')&(credits['total_income'].isna()), 'total_income'] = businessman_employee_income_avg

# проверка корректности замены пропущенных значений
credits[credits['total_income'].isna()].groupby('income_type')['income_type'].count().sort_values(ascending=False)

Series([], Name: income_type, dtype: int64)

С пропущенными значениями разобрались, теперь изучим остальные столбцы

In [17]:
# подсчет уникальных значений в каждом столбце датафрейма и вывод на экран для выявления возможных аномальных значений
for index in credits:
    display(credits[index].value_counts().sort_index(ascending=False))

 20       76
 5         9
 4        41
 3       330
 2      2055
 1      4818
 0     14149
-1        47
Name: children, dtype: int64

401755.400475    1
401715.811749    1
401675.093434    1
401674.466633    1
401663.850046    1
                ..
34.701045        1
33.520665        1
30.195337        1
24.240695        1
24.141633        1
Name: days_employed, Length: 19351, dtype: int64

75      1
74      6
73      8
72     33
71     58
70     65
69     85
68     99
67    167
66    183
65    194
64    265
63    269
62    352
61    355
60    377
59    444
58    461
57    460
56    487
55    443
54    479
53    459
52    484
51    448
50    514
49    508
48    538
47    480
46    475
45    497
44    547
43    513
42    597
41    607
40    609
39    573
38    598
37    537
36    555
35    617
34    603
33    581
32    510
31    560
30    540
29    545
28    503
27    493
26    408
25    357
24    264
23    254
22    183
21    111
20     51
19     14
0     101
Name: dob_years, dtype: int64

ученая степень             4
среднее                13750
неоконченное высшее      668
начальное                250
высшее                  4718
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Среднее                  711
СРЕДНЕЕ                  772
Неоконченное высшее       47
Начальное                 15
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Высшее                   268
ВЫСШЕЕ                   274
Name: education, dtype: int64

4        6
3      282
2      744
1    15233
0     5260
Name: education_id, dtype: int64

...

XNA        1
M       7288
F      14236
Name: gender, dtype: int64

студент                1
сотрудник          11119
предприниматель        2
пенсионер           3856
компаньон           5085
госслужащий         1459
в декрете              1
безработный            2
Name: income_type, dtype: int64

1     1741
0    19784
Name: debt, dtype: int64

2.265604e+06    1
2.200852e+06    1
1.726276e+06    1
1.715018e+06    1
1.711309e+06    1
               ..
2.189561e+04    1
2.169510e+04    1
2.136765e+04    1
2.120528e+04    1
2.066726e+04    1
Name: total_income, Length: 19355, dtype: int64

сыграть свадьбу                           774
строительство собственной недвижимости    635
строительство недвижимости                620
строительство жилой недвижимости          626
сделка с подержанным автомобилем          489
сделка с автомобилем                      455
свой автомобиль                           480
свадьба                                   797
ремонт жилью                              612
профильное образование                    436
приобретение автомобиля                   462
получение образования                     443
получение дополнительного образования     447
получение высшего образования             426
покупка своего жилья                      620
покупка недвижимости                      624
покупка коммерческой недвижимости         664
покупка жилья для семьи                   641
покупка жилья для сдачи                   653
покупка жилья                             647
покупка жилой недвижимости                607
операции со своей недвижимостью   

В столбце "Количество детей в семье" обнаружены два артефакта: 76 семей с 20-ю детьми и 47 семей с отрицательным количеством детей, т.е. всего 0,57% ошибочных данных по детям от общего количества клиентов.  
Предположим, что такие ошибки обусловлены человеческим фактором. Вероятно это опечатки, и в тех ячейках, где указано 20 детей, 0 лишний, а где -1 имели ввиду 1.  
Поскольку ошибки составляют менее 1%, можно было их проигнорировать, но данные из этого столбца нам понадобятся для исследования, поэтому сделаем замену, исходя из предположения, что ошибки были опечатками. Заменим 20 на 2, а -1 на 1.

In [18]:
# замена аномальных значений в стоблце "Количество детей в семье"
credits.loc[credits['children'] == 20, 'children'] = 2
credits.loc[credits['children'] == -1, 'children'] = 1

# проверка выполненной замены и вывод уникальных значений столбца "Количество детей в семье"
credits['children'].value_counts().sort_index()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь данные в столбце "Количество детей в семье" можно использовать для дальнейшего исследования.

**Вывод**

Предобработка данных обнаружила несколько проблем в данных:
1. В столбце "Общий трудовой стаж в днях" много ошибочных значений, например, стаж 401 674,85 дней, т.е. около 1100 лет. Вероятнее всего, это техническая ошибка, которая возникла при переносе или копировании данных.  
2. В колонке "Возраст клиента в годах" есть значения, равные нулю. Таких значений меньше 1%, поэтому оставим их без изменений. 
3. В столбце "Пол клиента" одно значение неопределено.
Данные из этих трех столбцов нам не понадобятся для исследования, поэтому менять ничего не будем.

А вот данные из колонки "Ежемесячный доход" будем использовать при проверке гипотезы о зависимости между уровнем дохода и возвратом кредита в срок, поэтому нам очень важно правильно их обработать. Средний доход отличается по группам занятости, поэтому мы посчитали медианные значения для каждого типа занятости в таблице с пропусками. Затем заменили пропуски на медианные значения в каждой группе, это поможет сделать более точные расчеты при категоризации клиентов по уровню дохода.  

В столбце "Уровень образования клиента" мы выявили множество дубликатов значений, которые исправим по ходу исследования.

В колонке "Ежемесячный доход" есть аномально большие значения, поменяем тип данных в этом столбце с вещественного на целочисленный, так данные будут занимать меньше места.

### Тип данных

In [19]:
# замена вещественного типа данных в столбце "Ежемесячный доход" на целочисленный тип данных
credits['total_income'] = credits['total_income'].astype('int')
credits.info()
display(credits.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Для замены вещественного типа данных на целочисленный в столбце "Ежемесячный доход" выбрали метод `astype('int')`. Это универсальный метод, с помощью которого можно пробовать перейти от одного типа данных к другому.

### Дубликаты

In [20]:
# подсчёт уникальных значений в столбце "Уровень образования клиента"
credits['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [21]:
# перевод символов к нижнему регистру в столбце "Уровень образования клиента"
credits['education_lowercase'] = credits['education'].str.lower()

# подсчёт уникальных значений после перевода символов в нижний регистр в столбце "Уровень образования клиента"
credits['education_lowercase'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

**Вывод**

В столбце "Уровень образования клиента" мы обнаружили неявные дубликаты - некоторые значения записаны с разным регистром букв. Скорее всего такие дубликаты вызваны человеческим фактором - одни клиенты заполняли анкеты строчными буквами, другие - прописными.

С помощью метода `.value_counts()` мы выяснили количество дубликатов. 

Методом `.str.lower()` - привели все символы в столбце к нижнему регистру и таким образом избавились от неявных дубликатов, чтобы сделать исследование более точным. 

Из полученных данных мы видим, что большинство людей, которые берут кредиты - это клиенты со средним образованием.

### Лемматизация

In [22]:
# выделение лемм в значениях столбца "Цель получения кредита"
from pymystem3 import Mystem
from collections import Counter

unique_purpose = credits['purpose'].unique()
lemmas_list = []
m = Mystem()

for purpose in unique_purpose:
    lemmas = ''.join(m.lemmatize(purpose)).strip()
    lemmas_list.append(lemmas)
display(Counter(lemmas_list))

Counter({'покупка жилье': 1,
         'приобретение автомобиль': 1,
         'дополнительный образование': 1,
         'сыграть свадьба': 1,
         'операция с жилье': 1,
         'образование': 1,
         'на проведение свадьба': 1,
         'покупка жилье для семья': 1,
         'покупка недвижимость': 1,
         'покупка коммерческий недвижимость': 1,
         'покупка жилой недвижимость': 1,
         'строительство собственный недвижимость': 1,
         'недвижимость': 1,
         'строительство недвижимость': 1,
         'на покупка подержать автомобиль': 1,
         'на покупка свой автомобиль': 1,
         'операция с коммерческий недвижимость': 1,
         'строительство жилой недвижимость': 1,
         'жилье': 1,
         'операция со свой недвижимость': 1,
         'автомобиль': 2,
         'заниматься образование': 1,
         'сделка с подержанный автомобиль': 1,
         'получение образование': 1,
         'свадьба': 1,
         'получение дополнительный образование'

**Вывод**

Для лемматизации целей кредита мы составили список уникальных значений в столбце "Цель получения кредита". Затем привели все слова в списке к базовой форме с учетом контекста.  
На основе полученных данных разделим цели кредита на 4 категории: 
-  автомобиль, 
- недвижимость, 
- образование, 
- свадьба.

### Категоризация данных

In [23]:
# создание категорий для столбца с целями получения кредита
def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ):
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    return 'свадьба'
credits['purpose_category'] = credits.apply(create_category_purpose, axis=1)
display(credits.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба


In [24]:
# подсчёт квартилей для ежемесячного дохода 
first_quartile = credits['total_income'].quantile(.25)
second_quartile = credits['total_income'].quantile(.5)
third_quartile = credits['total_income'].quantile(.75)
print('Первый квартиль:', first_quartile)
print('Второй квартиль:', second_quartile)
print('Третий квартиль:', third_quartile)

Первый квартиль: 107798.0
Второй квартиль: 151931.0
Третий квартиль: 202417.0


Расчёт квартилей поможет нам более точно определить категории клиентов по уровню дохода.

In [25]:
# создание категорий для столбца "Ежемесячный доход" на основе рассчитанных квартилей
def create_category_income(income):
    if income <= first_quartile:
        return 'бедные'
    if income <= second_quartile:
        return 'выше бедности'
    if income <= third_quartile:
        return 'средний класс'
    return 'богатые и сверхбогатые'
credits['income_category'] = credits['total_income'].apply(create_category_income)
display(credits.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_category,income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,недвижимость,богатые и сверхбогатые
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль,выше бедности
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,недвижимость,выше бедности
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование,богатые и сверхбогатые
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба,средний класс


In [26]:
# создание категорий для семейного статуса: 
def create_category_family(family_status_id):
    if (family_status_id == 0) or (family_status_id == 1):
        return 'семейные'
    return 'одинокие'
credits['family_status_category'] = credits['family_status_id'].apply(create_category_family)
display(credits.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_category,income_category,family_status_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,недвижимость,богатые и сверхбогатые,семейные
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль,выше бедности,семейные
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,недвижимость,выше бедности,семейные
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование,богатые и сверхбогатые,семейные
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба,средний класс,семейные
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,высшее,недвижимость,богатые и сверхбогатые,семейные
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,высшее,недвижимость,богатые и сверхбогатые,семейные
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,среднее,образование,выше бедности,семейные
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,высшее,свадьба,бедные,семейные
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,среднее,недвижимость,выше бедности,семейные


In [27]:
# создание категорий по наличию или отсутствию детей 
def create_category_children(children):
    if children == 0:
        return 'детей нет'
    if children == 1:
        return 'семья с одним ребёнком'
    if children == 2:
        return 'семья с двумя детьми'
    return 'многодетная семья'
credits['children_category'] = credits['children'].apply(create_category_children)
display(credits.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_category,income_category,family_status_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,недвижимость,богатые и сверхбогатые,семейные,семья с одним ребёнком
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль,выше бедности,семейные,семья с одним ребёнком
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,недвижимость,выше бедности,семейные,детей нет
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование,богатые и сверхбогатые,семейные,многодетная семья
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба,средний класс,семейные,детей нет


In [28]:
# создание категорий по наличию задолженности по возврату кредитов
def create_category_debt(debt):
    if debt != 0:
        return 'имел задолженность'
    return 'не имел задолженность'
credits['debt_category'] = credits['debt'].apply(create_category_debt)
display(credits.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_category,income_category,family_status_category,children_category,debt_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,недвижимость,богатые и сверхбогатые,семейные,семья с одним ребёнком,не имел задолженность
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,автомобиль,выше бедности,семейные,семья с одним ребёнком,не имел задолженность
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,недвижимость,выше бедности,семейные,детей нет,не имел задолженность
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,образование,богатые и сверхбогатые,семейные,многодетная семья,не имел задолженность
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,свадьба,средний класс,семейные,детей нет,не имел задолженность
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,высшее,недвижимость,богатые и сверхбогатые,семейные,детей нет,не имел задолженность
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,высшее,недвижимость,богатые и сверхбогатые,семейные,детей нет,не имел задолженность
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,среднее,образование,выше бедности,семейные,детей нет,не имел задолженность
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,высшее,свадьба,бедные,семейные,семья с двумя детьми,не имел задолженность
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,среднее,недвижимость,выше бедности,семейные,детей нет,не имел задолженность


**Вывод**

Для столбца с целями кредита выделили 4 категории: 
- автомобиль, 
- недвижимость,
- образование, 
- свадьба.

Для столбца "Уровень дохода" мы рассчитали квартили, по которым выделили 4 категории экономического состояния клиентов:
- бедные (если уровень дохода меньше первого квартиля), 
- выше бедности (если уровень дохода меньше второго квартиля),
- средний класс (если уровень дохода меньше третьего квартиля), 
- богатые и сверхбогатые для всех остальных случаев.

Клиентов с разным семейным статусом объединили в две группы:
- семейные (клиенты со статусом "гражданский брак" и "женат/замужем"),
- одинокие (клиенты со статусом "не женат/не замужем", "в разводе", "вдова/вдовец").

По наличию детей получилось 4 категории: 
- детей нет, 
- семья с одним ребёнком, 
- семья с двумя детьми, 
- многодетная семья.

По наличию задолженности также получили две категории:
- имел задолженность, 
- не имел задолженность.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
# сводная таблица по наличию детей и задолженности по возврату кредита
data_pivot_children = credits.pivot_table(
    index = 'debt_category', 
    columns = 'children_category', 
    values = 'debt', 
    aggfunc = 'count')
display(data_pivot_children.sort_values(by='debt_category', ascending=True))

children_category,детей нет,многодетная семья,семья с двумя детьми,семья с одним ребёнком
debt_category,,,,
имел задолженность,1063,31,202,445
не имел задолженность,13086,349,1929,4420


**Вывод**

Если сравнивать клиентов с детьми и без детей, можно сделать такие выводы:
- клиенты без детей чаще берут кредиты и чаще задерживают выплаты,
- многодетные семьи стараются избежать кредитных зависимостей и крайне редко задерживают выплаты по имеющимся кредитам.

Значит данные говорят, что зависимость между наличием детей и возвратом кредита в срок имеется - клиенты с детьми склонны не иметь задолженности в виде кредита и по выплате текущих кредитов тоже.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
# сводная таблица по семейному положению и наличию задолженности по возврату кредита
data_pivot_family = credits.pivot_table(
    index = 'debt_category', 
    columns = 'family_status_category', 
    values = 'debt', 
    aggfunc = 'count')
display(data_pivot_family.sort_values(by='debt_category', ascending=True))

family_status_category,одинокие,семейные
debt_category,,
имел задолженность,422,1319
не имел задолженность,4546,15238


**Вывод**

Данные показывают как семейное положение влияет на возврат кредита в срок:
- одинокие клиенты реже берут кредиты, чем пары,
- семейные пары чаще выплачивают кредиты в срок.

Таким образом, получается, что среди должников меньше всего одиноких клиентов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
# сводная таблица по уровню дохода и наличию задолженности по возврату кредита
data_pivot_income = credits.pivot_table(
    index = 'debt_category', 
    columns = 'income_category', 
    values = 'debt', 
    aggfunc = 'count')
display(data_pivot_income.sort_values(by='debt_category', ascending=True))

income_category,бедные,богатые и сверхбогатые,выше бедности,средний класс
debt_category,,,,
имел задолженность,427,345,475,494
не имел задолженность,4955,4564,4906,5359


**Вывод**

Данные показывают, что кредиты берут клиенты с разным уровнем дохода, в т.ч. "богатые и сверхбогатые", но при этом клиенты из категории "средний класс" чаще задерживают выплаты по обязательствам перед банком.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
# сводная таблица по целям кредита и наличию задолженности по возврату кредита
data_pivot_purpose = credits.pivot_table(
    index = 'debt_category', 
    columns = 'purpose_category', 
    values = 'debt', 
    aggfunc = 'count')
display(data_pivot_purpose.sort_values(by='debt_category', ascending=True))

purpose_category,автомобиль,недвижимость,образование,свадьба
debt_category,,,,
имел задолженность,403,782,370,186
не имел задолженность,3912,10058,3652,2162


**Вывод**

Если сравнивать разные цели кредитов, можно сделать такие выводы:
1. Кредиты на недвижимость берут чаще, чем для других целей, но и задерживают выплаты по таким кредитам чаще, чем по остальным, 
2. Меньше всего кредиты берут для проведения свадеб и при этом стараются вернуть в срок.

Таким образом, получается, что для приобретения материальных ценностей (автомобили и недвижимость) кредиты берут чаще, но выплачивают без задержек кредиты, нацеленные на нематериальные потребности (образование и свадьбы).

## Итоги исследования

Данные статистики о платёжеспособности клиентов мы получили от кредитного отдела банка. Поскольку о качестве данных ничего не было известно, мы провели обзор данных, чтобы выделить явные неточности.

Мы проверили данные на ошибки и оценили их влияние на исследование. Предобработка обнаружила 5 проблем в данных:
1. пропуски значений,
2. некорректный тип данных,
3. аномальные значения, которые не отражают действительность,
4. опечатки, вызванные человеческим фактором,
5. неявные дубликаты.

Самые критичные ошибки данных мы исправили:
* заполнили пропущенные значения в столбце "Ежемесячный доход" медианным значением для каждого типа занятости клиента, чтобы точнее разделить клиентов на категории и уровню дохода;
* поменяли тип данных с вещественного на целочисленный для столбца "Ежемесячный доход"; 
* заменили опечатки на наиболее подходящие значения для каждого случая в столбце "Количество детей в семье" для более точного исследования;
* удалили неявные дубликаты в столбце "Уровень образования".

Таким образом, исследование проводили в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы.

По результатам исследования статистики о платёжеспособности клиентов банка мы установили, что:
* при наличии детей клиенты стремятся не влезать в долги и вовремя возвращать заемные средства,
* потребности клиентов, имеющих семью, растут и им чаще приходится прибегать к помощи банка, чем одиноким людям,
* кредитными средствами пользуются клиенты с разным уровнем доходов - от бедных до богатых,
* для решения жилищных вопросов чаще всего не обойтись без кредита.

Подведём итоги нашей работы - несколько рекомендаций для кредитного отдела банка:
* Обратить внимание на сбор данных о клиентах, потому что при наличии ошибок во входных данных даже правильный алгоритм работы может привести к неверным результатам. Например, большинство пропущенных значений в столбце "Ежемесячный доход" были среди сотрудников банка. Аномальные данные в столбце "Общий трудовой стаж в днях" невозможно скорректировать. Для исследования с другими целями, в котором бы учитывался стаж клиента, не удалось бы использовать такие данные.
* Сделать акцент на получение подтверждающих документов о доходах клиентов. Возможно некоторые из клиентов вовсе не имеют постоянного дохода и могут задерживать выплаты по кредиту. 